# Assignment 2.1 - Data Lake

## Dependency Setup

In [1]:
!python --version

Python 3.11.11


In [2]:
# Combining all installations into one cell
!pip install --disable-pip-version-check -q pip --upgrade > /dev/null
!pip install --disable-pip-version-check -q wrapt --upgrade > /dev/null
!pip install --disable-pip-version-check -q awscli boto3
!pip install --disable-pip-version-check -q sagemaker
!pip install --disable-pip-version-check -q smdebug
!pip install --disable-pip-version-check -q sagemaker-experiments
!pip install --disable-pip-version-check -q PyAthena
!pip install --disable-pip-version-check -q awswrangler
!pip install --disable-pip-version-check -q matplotlib
!pip install --disable-pip-version-check -q seaborn

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.1.1 requires nvidia-ml-py3==7.352.0, which is not installed.
aiobotocore 2.13.3 requires botocore<1.34.163,>=1.34.70, but you have botocore 1.36.2 which is incompatible.
amazon-sagemaker-sql-magic 0.1.3 requires sqlparse==0.5.0, but you have sqlparse 0.5.3 which is incompatible.
autogluon-core 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-core 1.1.1 requires scipy<1.13,>=1.5.4, but you have scipy 1.14.1 which is incompatible.
autogluon-features 1.1.1 requires scikit-learn<1.4.1,>=1.3.0, but you have scikit-learn 1.5.2 which is incompatible.
autogluon-multimodal 1.1.1 requires jsonschema<4.22,>=4.18, but you have jsonschema 4.23.0 which is incompatible.
autogluon-multimodal 1.1.1 requires omegaconf<2.3.0,>=2.1.1, but you have omega

In [ ]:
!pip list

In [4]:
!conda install -y zip

Channels:
 - conda-forge
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - zip


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    libpng-1.6.45              |       h943b412_0         283 KB  conda-forge
    torchvision-0.19.1         |cpu_py311h8c76117_3        10.0 MB  conda-forge
    ------------------------------------------------------------
                                           Total:        10.3 MB

The following NEW packages will be INSTALLED:

  zip                conda-forge/linux-64::zip-3.0-hd590300_3 

The following packages will be UPDATED:

  amazon-sagemaker-~                     3.1.7-pyhd8ed1ab_0 --> 3.1.8-pyhd8ed1ab_0 
  amazon_sagemaker_~                    0.1.13-pyhd8ed1ab_0 --> 0.1.14-pyhd8ed1ab_0 
  aws-glue-sessions                      1.0.7-pyhd8ed1ab_0 --> 1.0.8-pyhd8ed1ab_0

### Dependency Summary

In [5]:
setup_dependencies_passed = True

In [6]:
%store setup_dependencies_passed

Stored 'setup_dependencies_passed' (bool)


In [7]:
%store

Stored variables and their in-db values:
s3_private_path_csv                   -> 's3://sagemaker-us-east-1-203012117619/assignment-
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


### Dependency Imports

In [103]:
# AWS Imports
import boto3
from botocore.client import ClientError
import sagemaker
from pyathena import connect
import awswrangler as wr

# Data Transformation Imports
import pandas as pd
from io import StringIO

# Misc Imports
from IPython.display import display, HTML

## S3 Initialization

In [9]:
session = boto3.session.Session()
region = session.region_name
sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

s3 = boto3.Session().client(service_name="s3", region_name=region)

In [10]:
setup_s3_bucket_passed = False

In [11]:
print("Default bucket: {}".format(bucket))

Default bucket: sagemaker-us-east-1-203012117619


### Bucket Verification

In [12]:
response = None

try:
    response = s3.head_bucket(Bucket=bucket)
    print(response)
    setup_s3_bucket_passed = True
except ClientError as e:
    print("[ERROR] Cannot find bucket {} in {} due to {}.".format(bucket, response, e))

{'ResponseMetadata': {'RequestId': '8Z5RE2061BKFHNG0', 'HostId': 's/+Zo9LR5eZFac0SgezBZHMgTUi1rqzKioXcclwz165ssZD9MHK98XK3kMMWbpeTsWDLow4Wv0c=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 's/+Zo9LR5eZFac0SgezBZHMgTUi1rqzKioXcclwz165ssZD9MHK98XK3kMMWbpeTsWDLow4Wv0c=', 'x-amz-request-id': '8Z5RE2061BKFHNG0', 'date': 'Mon, 20 Jan 2025 04:20:34 GMT', 'x-amz-bucket-region': 'us-east-1', 'x-amz-access-point-alias': 'false', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'BucketRegion': 'us-east-1', 'AccessPointAlias': False}


In [13]:
%store setup_s3_bucket_passed

Stored 'setup_s3_bucket_passed' (bool)


In [14]:
%store

Stored variables and their in-db values:
s3_private_path_csv                   -> 's3://sagemaker-us-east-1-203012117619/assignment-
setup_dependencies_passed             -> True
setup_s3_bucket_passed                -> True


## Copy Dataset to S3 Bucket

In [15]:
try:
    setup_dependencies_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++")

In [16]:
print(setup_dependencies_passed)

True


In [17]:
%store -r setup_s3_bucket_passed

In [18]:
try:
    setup_s3_bucket_passed
except NameError:
    print("+++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++")

In [19]:
print(setup_s3_bucket_passed)

True


In [20]:
if not setup_dependencies_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup Dependencies.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
if not setup_s3_bucket_passed:
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL NOTEBOOKS IN THE SETUP FOLDER FIRST. You are missing Setup S3 Bucket.")
    print("+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++")

In [21]:
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

### Setting S3 Destination Location

In [22]:
s3_private_path_csv = "s3://{}/assignment-2/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-203012117619/assignment-2/csv


In [23]:
%store s3_private_path_csv

Stored 's3_private_path_csv' (str)


### Copying Data from Local Directory to S3

In [35]:
!aws s3 cp "dataset.csv" $s3_private_path_csv/

upload: ./dataset.csv to s3://sagemaker-us-east-1-203012117619/assignment-2/csv/dataset.csv


### Listing Files in our Bucket

In [24]:
print(s3_private_path_csv)

s3://sagemaker-us-east-1-203012117619/assignment-2/csv


In [25]:
!aws s3 ls $s3_private_path_csv/

2025-01-19 19:48:31    1048576 dataset.csv


In [26]:
display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/sagemaker-{}-{}/assignment-2/?region={}&tab=overview">S3 Bucket</a></b>'.format(
            region, account_id, region
        )
    )
)

## Creating Athena Schema

In [27]:
ingest_create_athena_db_passed = False

In [29]:
database_name = "dsoaws"

In [30]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [31]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [32]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)

CREATE DATABASE IF NOT EXISTS dsoaws


In [33]:
pd.read_sql(statement, conn)

/tmp/ipykernel_123/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


### DB Verification

In [34]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_123/3999478089.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws


In [35]:
if database_name in df_show.values:
    ingest_create_athena_db_passed = True

In [36]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


## Registering Data with Athena

### Creating a Table from CSV file

In [57]:
# Set Athena parameters
database_name = "dsoaws"
table_name_csv = "assignment_2_dataset_csv"

In [58]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [84]:
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
    index_col INT,
    track_id STRING,
    artists STRING,
    album_name STRING,
    track_name STRING,
    popularity INT,
    duration_ms INT,
    explicit BOOLEAN,
    danceability FLOAT,
    energy FLOAT,
    key INT,
    loudness FLOAT,
    mode INT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INT,
    track_genre STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_private_path_csv
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS dsoaws.assignment_2_dataset_csv(
    index_col INT,
    track_id STRING,
    artists STRING,
    album_name STRING,
    track_name STRING,
    popularity INT,
    duration_ms INT,
    explicit BOOLEAN,
    danceability FLOAT,
    energy FLOAT,
    key INT,
    loudness FLOAT,
    mode INT,
    speechiness FLOAT,
    acousticness FLOAT,
    instrumentalness FLOAT,
    liveness FLOAT,
    valence FLOAT,
    tempo FLOAT,
    time_signature INT,
    track_genre STRING
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-203012117619/assignment-2/csv'
TBLPROPERTIES ('skip.header.line.count'='1')


### Table Verification

In [85]:
pd.read_sql(statement, conn)

/tmp/ipykernel_123/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [86]:
statement = "SHOW TABLES in {}".format(database_name)

df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_123/2201015668.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,assignment_2_dataset_csv


## SQL Queries

### List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [88]:
SQLQ1 = f"""SELECT artists,track_name, popularity FROM {database_name}.{table_name_csv} WHERE popularity >= 99;"""
SQLQ1_show = pd.read_sql(SQLQ1,conn)
SQLQ1_show.head(10)

/tmp/ipykernel_123/3351755692.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQLQ1_show = pd.read_sql(SQLQ1,conn)


,artists,track_name,popularity


### List artists with an average popularity of 92

In [89]:
SQLQ2 = f"""
SELECT 
    artists, 
    AVG(popularity) AS average_popularity 
FROM 
    {database_name}.{table_name_csv} 
GROUP BY 
    artists 
HAVING 
    AVG(popularity) = 92;
"""

SQLQ2_show = pd.read_sql(SQLQ2,conn)
SQLQ2_show.head(10)

/tmp/ipykernel_123/1258578713.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQLQ2_show = pd.read_sql(SQLQ2,conn)


,artists,average_popularity


### List the Top 10 most energetic genres

In [90]:
SQLQ3 = f"""SELECT 
    track_genre, 
    AVG(energy) AS avg_energy 
FROM 
    {database_name}.{table_name_csv} 
GROUP BY 
    track_genre 
ORDER BY 
    avg_energy DESC 
LIMIT 10;"""

SQLQ3_show = pd.read_sql(SQLQ3,conn)
SQLQ3_show.head(10)

/tmp/ipykernel_123/4072621192.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQLQ3_show = pd.read_sql(SQLQ3,conn)


,track_genre,avg_energy
0,0.104,54.000000
1,black-metal,0.879181
2,alt-rock,0.755122
3,alternative,0.721058
4,afrobeat,0.702747
5,anime,0.677280
6,4,0.540275
7,1,0.452000
8,3,0.440323
9,acoustic,0.439168


### How many tracks is Bad Bunny on?

In [82]:
SQLQ4 = f"""SELECT 
    COUNT(*) AS track_count 
FROM 
    {database_name}.{table_name_csv} 
WHERE 
    artists LIKE '%Bad Bunny%';"""

SQLQ4_show = pd.read_sql(SQLQ4,conn)
SQLQ4_show.head(10)

/tmp/ipykernel_123/965045587.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQLQ4_show = pd.read_sql(SQLQ4,conn)


,track_count
0,0


### Show the top 10 genres in terms of popularity sorted by their most popular track

In [83]:
SQLQ5 = f"""SELECT 
    track_genre, 
    MAX(popularity) AS max_popularity 
FROM 
    {database_name}.{table_name_csv}
GROUP BY 
    track_genre 
ORDER BY 
    max_popularity DESC 
LIMIT 10;"""

SQLQ5_show = pd.read_sql(SQLQ5,conn)
SQLQ5_show.head(10)

/tmp/ipykernel_123/1114623197.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  SQLQ5_show = pd.read_sql(SQLQ5,conn)


,track_genre,max_popularity
0,alternative,93.0
1,alt-rock,93.0
2,ambient,84.0
3,anime,83.0
4,acoustic,82.0
5,afrobeat,75.0
6,black-metal,58.0
7,129.491,NaN
8,125.925,NaN
9,track_genre,NaN


## Dataframe Queries

In [98]:
response = s3.get_object(Bucket=bucket,Key="assignment-2/csv/dataset.csv")

{'ResponseMetadata': {'RequestId': 'QHT8Y125PHA05SHC',
  'HostId': '49SUkuf1B7kbiKo3l+J7yG6y4Ba1K50SpPCkCxY4Gxfol7w8TBhstp6+Qhz0kmCkLHhOHp/0s8Y=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': '49SUkuf1B7kbiKo3l+J7yG6y4Ba1K50SpPCkCxY4Gxfol7w8TBhstp6+Qhz0kmCkLHhOHp/0s8Y=',
   'x-amz-request-id': 'QHT8Y125PHA05SHC',
   'date': 'Mon, 20 Jan 2025 05:24:21 GMT',
   'last-modified': 'Sun, 19 Jan 2025 19:48:31 GMT',
   'etag': '"42ff51fa674f918e1cec174245acbda2"',
   'x-amz-checksum-crc32': 'tT/7tA==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'text/csv',
   'content-length': '1048576',
   'server': 'AmazonS3'},
  'ChecksumAlgorithm': 'crc32',
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2025, 1, 19, 19, 48, 31, tzinfo=tzutc()),
 'ContentLength': 1048576,
 'ETag': '"42ff51fa674f918e1cec174245acbda2"',
 'ChecksumCRC32': 'tT/7tA==',
 'ChecksumType': 'FULL

In [104]:
csv_content = response['Body'].read().decode('utf-8')

### Cleaning

In [ ]:
df = pd.read_csv(StringIO(csv_content))
df = df.drop(['Unnamed: 0'], axis=1)

In [106]:
df.head()

,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73.0,230666.0,False,0.676,0.4610,1.0,-6.746,0.0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4.0,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55.0,149610.0,False,0.420,0.1660,1.0,-17.235,1.0,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4.0,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57.0,210826.0,False,0.438,0.3590,0.0,-9.734,1.0,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4.0,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71.0,201933.0,False,0.266,0.0596,0.0,-18.515,1.0,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3.0,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82.0,198853.0,False,0.618,0.4430,2.0,-9.681,1.0,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4.0,acoustic


### Reuploading Cleaned CSV File 

csv_buffer = StringIO()
df.to_csv(csv_buffer, index=False)

# Step 4: Reupload the CSV to S3
s3.put_object(Bucket=bucket, Key='assignment-2/csv/cleaned-dataset.csv', Body=csv_buffer.getvalue())
print("File successfully updated and uploaded!")

### List artist, track_name, and popularity for songs that have a popularity greater than or equal to 99

In [112]:
result = df[df['popularity'] >= 99][['artists', 'track_name', 'popularity']]
print(result)

Empty DataFrame
Columns: [artists, track_name, popularity]
Index: []


### List artists with an average popularity of 92

In [113]:
grouped = df.groupby('artists')['popularity'].mean().reset_index()

# Filter for artists with an average popularity of 92
result = grouped[grouped['popularity'] == 92]
print(result)

Empty DataFrame
Columns: [artists, popularity]
Index: []


### List the Top 10 most energetic genres

In [114]:
# Group by genre and calculate the average energy
grouped = df.groupby('track_genre')['energy'].mean().reset_index()

# Sort by average energy in descending order and get the top 10
result = grouped.sort_values(by='energy', ascending=False).head(10)
print(result)

   track_genre    energy
6  black-metal  0.877573
2     alt-rock  0.754173
3  alternative  0.720030
1     afrobeat  0.702812
5        anime  0.674108
0     acoustic  0.435368
4      ambient  0.237162


### How many tracks is Bad Bunny on?

In [115]:
# Filter rows where 'Bad Bunny' is in the artists column and count them
result = df[df['artists'].str.contains('Bad Bunny', na=False)]['track_id'].count()
print(f"Bad Bunny is on {result} tracks.")

Bad Bunny is on 0 tracks.


### Show the top 10 genres in terms of popularity sorted by their most popular track

In [118]:
# Group by genre and get the maximum popularity for each genre
grouped = df.groupby('track_genre')['popularity'].max().reset_index()

# Sort by the most popular track in descending order and get the top 10 genres
result = grouped.sort_values(by='popularity', ascending=False).head(10)
print(result)

   track_genre  popularity
2     alt-rock        93.0
3  alternative        93.0
4      ambient        84.0
5        anime        83.0
0     acoustic        82.0
1     afrobeat        75.0
6  black-metal        58.0
